# Обработка данных Superstore и загрузка в БД в MS SQL Server

## 1. Импорт датасета, проверка дубликатов, преобразование данных и подготовка к экспорту в БД

In [1]:
# Импорт библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from chardet.universaldetector import UniversalDetector
import datetime
from sqlalchemy import create_engine
import urllib
#import string

In [2]:
# Функция для определения кодировки
def encode_detector(path):
    detector = UniversalDetector()
    with open(path, 'rb') as fh:
        for line in fh:
            detector.feed(line)
            if detector.done:
                break
        detector.close()
    print (detector.result)
    return detector.result["encoding"]

    #type(detector.result)

encod_res = encode_detector(path = '../data/superstore-sample.csv')

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


In [3]:
df_orig = pd.read_csv('../data/superstore-sample.csv', sep = ',', encoding = encod_res)
df_orig.info()
df_orig.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

Row ID           9994
Order ID         5009
Order Date       1237
Ship Date        1334
Ship Mode           4
Customer ID       793
Customer Name     793
Segment             3
Country             1
City              531
State              49
Postal Code       631
Region              4
Product ID       1862
Category            3
Sub-Category       17
Product Name     1850
Sales            5825
Quantity           14
Discount           12
Profit           7287
dtype: int64

In [4]:
#df_renamed = df_orig.rename(columns = {'Row ID': 'row_id', 'Order ID': 'order_id'})
df_orig.columns

def rename_col_func(df):
    new_col = []
    for i in df.columns.array:
        i = i.lower().replace(" ", "_")
        new_col.append(i)
    df.columns = new_col
    return df

df_renamed = rename_col_func(df_orig)
df_maybe_dup = df_renamed.drop('row_id', axis = 1)
df_maybe_dup.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_id       9994 non-null   object 
 1   order_date     9994 non-null   object 
 2   ship_date      9994 non-null   object 
 3   ship_mode      9994 non-null   object 
 4   customer_id    9994 non-null   object 
 5   customer_name  9994 non-null   object 
 6   segment        9994 non-null   object 
 7   country        9994 non-null   object 
 8   city           9994 non-null   object 
 9   state          9994 non-null   object 
 10  postal_code    9994 non-null   int64  
 11  region         9994 non-null   object 
 12  product_id     9994 non-null   object 
 13  category       9994 non-null   object 
 14  sub-category   9994 non-null   object 
 15  product_name   9994 non-null   object 
 16  sales          9994 non-null   float64
 17  quantity       9994 non-null   int64  
 18  discount

In [5]:
#df.duplicated()
duplicates = df_maybe_dup[df_maybe_dup.duplicated(keep=False)]
duplicates

,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit
3405,US-2014-150119,4/23/2014,4/27/2014,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,Ohio,43229,East,FUR-CH-10002965,Furniture,Chairs,Global Leather Highback Executive Chair with P...,281.372,2,0.3,-12.0588
3406,US-2014-150119,4/23/2014,4/27/2014,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,Ohio,43229,East,FUR-CH-10002965,Furniture,Chairs,Global Leather Highback Executive Chair with P...,281.372,2,0.3,-12.0588


In [6]:
print(df_maybe_dup[['sales', 'quantity', 'profit']].sum())
#df.drop_duplicates(subset = None, keep ='first', inplace = True)
df = df_maybe_dup.drop_duplicates(keep='first')
print(df[['sales', 'quantity', 'profit']].sum())
print(df[['sales', 'quantity', 'profit']].sum() - df_maybe_dup[['sales', 'quantity', 'profit']].sum())

sales       2.297201e+06
quantity    3.787300e+04
profit      2.863970e+05
dtype: float64
sales       2.296919e+06
quantity    3.787100e+04
profit      2.864091e+05
dtype: float64
sales      -281.3720
quantity     -2.0000
profit       12.0588
dtype: float64


In [7]:
#df[df.duplicated(subset = ['order_id', 'product_id'], keep = False)]
duplicates = df.duplicated(subset = ['order_id', 'product_id']).sum()
print(duplicates)
df.info()

7
<class 'pandas.core.frame.DataFrame'>
Index: 9993 entries, 0 to 9993
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_id       9993 non-null   object 
 1   order_date     9993 non-null   object 
 2   ship_date      9993 non-null   object 
 3   ship_mode      9993 non-null   object 
 4   customer_id    9993 non-null   object 
 5   customer_name  9993 non-null   object 
 6   segment        9993 non-null   object 
 7   country        9993 non-null   object 
 8   city           9993 non-null   object 
 9   state          9993 non-null   object 
 10  postal_code    9993 non-null   int64  
 11  region         9993 non-null   object 
 12  product_id     9993 non-null   object 
 13  category       9993 non-null   object 
 14  sub-category   9993 non-null   object 
 15  product_name   9993 non-null   object 
 16  sales          9993 non-null   float64
 17  quantity       9993 non-null   int64  
 18  discount   

In [8]:
df[(df.duplicated(['order_id', 'order_date', 'ship_date', 'ship_mode', 
                     'customer_id', 'customer_name', 'segment', 'country', 
                     'city', 'state', 'postal_code', 'region',
                     'product_id', 'category', 'sub-category',
                     'product_name', 'discount'], keep=False)==True)]
# предполагается, что дубликатов ошибочных нет, при агрегации суммируются продажи, кол-во товаров, прибыль

,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit
350,CA-2016-129714,9/1/2016,9/3/2016,First Class,AB-10060,Adam Bellavance,Home Office,United States,New York City,New York,10009,East,OFF-PA-10001970,Office Supplies,Paper,Xerox 1881,24.560,2,0.0,11.5432
352,CA-2016-129714,9/1/2016,9/3/2016,First Class,AB-10060,Adam Bellavance,Home Office,United States,New York City,New York,10009,East,OFF-PA-10001970,Office Supplies,Paper,Xerox 1881,49.120,4,0.0,23.0864
430,US-2016-123750,4/15/2016,4/21/2016,Standard Class,RB-19795,Ross Baird,Home Office,United States,Gastonia,North Carolina,28052,South,TEC-AC-10004659,Technology,Accessories,Imation Secure+ Hardware Encrypted USB 2.0 Fla...,408.744,7,0.2,76.6395
431,US-2016-123750,4/15/2016,4/21/2016,Standard Class,RB-19795,Ross Baird,Home Office,United States,Gastonia,North Carolina,28052,South,TEC-AC-10004659,Technology,Accessories,Imation Secure+ Hardware Encrypted USB 2.0 Fla...,291.960,5,0.2,54.7425
1300,CA-2016-137043,12/23/2016,12/25/2016,Second Class,LC-17140,Logan Currie,Consumer,United States,Springfield,Virginia,22153,South,FUR-FU-10003664,Furniture,Furnishings,"Electrix Architect's Clamp-On Swing Arm Lamp, ...",572.760,6,0.0,166.1004
1301,CA-2016-137043,12/23/2016,12/25/2016,Second Class,LC-17140,Logan Currie,Consumer,United States,Springfield,Virginia,22153,South,FUR-FU-10003664,Furniture,Furnishings,"Electrix Architect's Clamp-On Swing Arm Lamp, ...",286.380,3,0.0,83.0502
3183,CA-2017-152912,11/9/2017,11/12/2017,Second Class,BM-11650,Brian Moss,Corporate,United States,Columbia,Maryland,21044,East,OFF-ST-10003208,Office Supplies,Storage,Adjustable Depth Letter/Legal Cart,1633.140,9,0.0,473.6106
3184,CA-2017-152912,11/9/2017,11/12/2017,Second Class,BM-11650,Brian Moss,Corporate,United States,Columbia,Maryland,21044,East,OFF-ST-10003208,Office Supplies,Storage,Adjustable Depth Letter/Legal Cart,544.380,3,0.0,157.8702
6498,CA-2015-103135,7/24/2015,7/28/2015,Standard Class,SS-20515,Shirley Schmidt,Home Office,United States,Louisville,Kentucky,40214,South,OFF-BI-10000069,Office Supplies,Binders,"GBC Prepunched Paper, 19-Hole, for Binding Sys...",135.090,9,0.0,62.1414
6500,CA-2015-103135,7/24/2015,7/28/2015,Standard Class,SS-20515,Shirley Schmidt,Home Office,United States,Louisville,Kentucky,40214,South,OFF-BI-10000069,Office Supplies,Binders,"GBC Prepunched Paper, 19-Hole, for Binding Sys...",90.060,6,0.0,41.4276


In [9]:
#Агрегирование данных
df_agg = df.groupby(['order_id', 'order_date', 'ship_date', 'ship_mode', 
                     'customer_id', 'customer_name', 'segment', 'country', 
                     'city', 'state', 'postal_code', 'region',
                     'product_id', 'category', 'sub-category',
                     'product_name', 'discount']).agg({'sales':'sum', 'quantity':'sum', 'profit':'sum'})
df_agg = df_agg.reset_index()
#df_agg
print('df_agg: ', len(df_agg))
print('duplicates deleted: ', len(df) - duplicates==len(df_agg))
print('df_agg columns: ', len(df_agg.columns))
print('duplicates if order_id: ', df_agg.duplicated(subset = 'order_id').sum())
print('duplicates if product_id: ', df_agg.duplicated(subset = 'product_id').sum())
print('duplicates if customer_id: ', df_agg.duplicated(subset = 'customer_id').sum())
print('duplicates if order_id and product_id: ', df_agg.duplicated(subset = ['order_id', 'product_id']).sum())
print('duplicates if order_id and customer_id: ', df_agg.duplicated(subset = ['order_id', 'customer_id']).sum())
print('duplicates if customer_id and product_id: ', df_agg.duplicated(subset = ['customer_id', 'product_id']).sum())
print('chosen primary key: order_id + product_id') #order_id, product_id - составной ПК для данной таблицы
df_agg.columns

df_agg:  9986
duplicates deleted:  True
df_agg columns:  20
duplicates if order_id:  4977
duplicates if product_id:  8124
duplicates if customer_id:  9193
duplicates if order_id and product_id:  0
duplicates if order_id and customer_id:  4977
duplicates if customer_id and product_id:  42
chosen primary key: order_id + product_id


Index(['order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id',
       'customer_name', 'segment', 'country', 'city', 'state', 'postal_code',
       'region', 'product_id', 'category', 'sub-category', 'product_name',
       'discount', 'sales', 'quantity', 'profit'],
      dtype='object')

In [10]:
#print(len(df_renamed))
#df_renamed.nunique()
print(len(df))
df.nunique()
df.info()

9993
<class 'pandas.core.frame.DataFrame'>
Index: 9993 entries, 0 to 9993
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_id       9993 non-null   object 
 1   order_date     9993 non-null   object 
 2   ship_date      9993 non-null   object 
 3   ship_mode      9993 non-null   object 
 4   customer_id    9993 non-null   object 
 5   customer_name  9993 non-null   object 
 6   segment        9993 non-null   object 
 7   country        9993 non-null   object 
 8   city           9993 non-null   object 
 9   state          9993 non-null   object 
 10  postal_code    9993 non-null   int64  
 11  region         9993 non-null   object 
 12  product_id     9993 non-null   object 
 13  category       9993 non-null   object 
 14  sub-category   9993 non-null   object 
 15  product_name   9993 non-null   object 
 16  sales          9993 non-null   float64
 17  quantity       9993 non-null   int64  
 18  discount

In [11]:
# Преобразуем тип данных для дат
df_agg['order_date'] = pd.to_datetime(df_agg['order_date'])
df_agg['ship_date'] = pd.to_datetime(df_agg['ship_date'])
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9986 entries, 0 to 9985
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_id       9986 non-null   object        
 1   order_date     9986 non-null   datetime64[ns]
 2   ship_date      9986 non-null   datetime64[ns]
 3   ship_mode      9986 non-null   object        
 4   customer_id    9986 non-null   object        
 5   customer_name  9986 non-null   object        
 6   segment        9986 non-null   object        
 7   country        9986 non-null   object        
 8   city           9986 non-null   object        
 9   state          9986 non-null   object        
 10  postal_code    9986 non-null   int64         
 11  region         9986 non-null   object        
 12  product_id     9986 non-null   object        
 13  category       9986 non-null   object        
 14  sub-category   9986 non-null   object        
 15  product_name   9986 n

In [12]:
df_for_check = df
def forgotten_unique_val_count(df_inp):
    unique_val = {}
    for i in list(df_inp.columns):
        unique_val.update({i: df_inp[i].nunique() - df_for_check[i].nunique()})
    return print(pd.DataFrame.from_dict(unique_val, orient = 'index', columns = ['forgotten_unique_val']))

def check_new_df(df_inp):
    x = print('rows: ', df_inp.shape[0])
    y = print('columns: ', df_inp.shape[1])
    a = print('duplicates: ', df_inp.duplicated().sum())
    check_val = {}
    for i in list(df_inp.columns):
        check_val.update({i: df_inp[i].nunique()})
    b = print(pd.DataFrame.from_dict(check_val, orient = 'index', columns = ['unique_values']))
    return x, y, a, b

In [13]:
# Создадим таблицу локаций доставки и добавим location_id для первичного ключа
df_locations = df_agg[['city', 'state', 'postal_code', 'country', 'region']].drop_duplicates(keep='first')
#print('duplicates: ', df_locations.duplicated().sum())
#print('city: ', df_locations['city'].nunique())
#print('state: ', df_locations['state'].nunique())
#print('postal_code: ', df_locations['postal_code'].nunique())
#print('region: ', df_locations['region'].nunique())
print('max postal codes for city: ', df_locations.groupby('city')['postal_code'].nunique().max())
print('max cities for postal code: ', df_locations.groupby('postal_code')['city'].nunique().max())
# Создаем location_id для каждой уникальной строки таблицы локаций доставки
df_locations['location_id'] = range(1, len(df_locations)+1)
#df_locations[df_locations.duplicated(subset = 'city', keep=False)].sort_values('city')
#df_locations[df_locations.duplicated(subset = 'postal_code', keep=False)]
check_new_df(df_locations)
print('location_id is unique: ', df_locations['location_id'].nunique() == len(df_locations))
forgotten_unique_val_count(df_locations.drop('location_id', axis = 1))
#df_locations


max postal codes for city:  6
max cities for postal code:  2
rows:  632
columns:  6
duplicates:  0
             unique_values
city                   531
state                   49
postal_code            631
country                  1
region                   4
location_id            632
location_id is unique:  True
             forgotten_unique_val
city                            0
state                           0
postal_code                     0
country                         0
region                          0


In [14]:
# Проверка на зависимость скидки от других характеристик
print(df_agg[['order_id', 'product_id', 'discount']].groupby(['order_id', 'product_id']).agg({'discount': 'nunique'}).query('discount > 1').count() == 0)
df_agg[['quantity', 'product_id', 'discount']].groupby(['quantity', 'product_id']).agg({'discount': 'nunique'})
# у одного заказа могут быть разные скидки для разных позиций товаров
# у одной позиции могут быть разные скидки в разных заказах
# одна скидка соответствует одной позиции товара в заказе
# В одном регионе по одному товару могут быть разные скидки
# Проверка соответствия скидки позиции товара и локации (штату)
df_agg.query('product_id == "FUR-BO-10001337"')
df_check_discount = df_agg[['state', 'product_id', 'discount']].groupby(['state', 'product_id']).agg({'discount': 'nunique'})
print(df_check_discount.count() == df_check_discount.query('discount == 1').count())
df_check_discount

discount    True
dtype: bool
discount    True
dtype: bool


discount
state     product_id               
Alabama   FUR-CH-10000513         1
          FUR-CH-10001854         1
          FUR-CH-10002647         1
          FUR-CH-10003774         1
          FUR-CH-10004853         1
...                             ...
Wisconsin TEC-PH-10002262         1
          TEC-PH-10003012         1
          TEC-PH-10004165         1
          TEC-PH-10004345         1
Wyoming   FUR-CH-10001215         1

[8242 rows x 1 columns]

In [15]:
# Из таблицы locations выделим таблицу states, нужно для нормализации БД с учётом будущей справочной таблицы скидок: promotions.
df_states = df_locations[['state', 'region', 'country']]
print('duplicates: ', df_states.duplicated().sum())
print('state: ', df_states['state'].nunique())
df_states = df_states.drop_duplicates(keep='first')

# Создаем state_id для каждой уникальной строки таблицы локаций доставки
df_states['state_id'] = range(1, len(df_states)+1)
#print(df_states)

#Добавим ключ state_id в таблицу locations и удалим столбцы state, region, country
df_locations = df_locations.merge(df_states[['state_id', 'state', 'region', 'country']], how = 'left')
print(df_locations['state_id'].nunique()==len(df_states))
df_locations = df_locations.drop(['state', 'region', 'country'], axis = 1)
#df_locations

duplicates:  583
state:  49
True


In [16]:
#Проверка созданных таблиц
check_new_df(df_locations)
forgotten_unique_val_count(df_locations.drop(['location_id', 'state_id'], axis=1))

check_new_df(df_states)
forgotten_unique_val_count(df_states.drop('state_id', axis=1))

rows:  632
columns:  4
duplicates:  0
             unique_values
city                   531
postal_code            631
location_id            632
state_id                49
             forgotten_unique_val
city                            0
postal_code                     0
rows:  49
columns:  4
duplicates:  0
          unique_values
state                49
region                4
country               1
state_id             49
         forgotten_unique_val
state                       0
region                      0
country                     0


In [17]:
# Добавим ключи location_id, state_id в датасет (для использования в других таблицах)
df_agg_merged = df_agg.merge(df_locations[['location_id', 'state_id', 'city', 'postal_code']], how = 'left')
print(df_agg_merged['location_id'].nunique()==len(df_locations))
print(df_agg_merged['state_id'].nunique()==len(df_states))
df_agg_merged

True
True


,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,...,product_id,category,sub-category,product_name,discount,sales,quantity,profit,location_id,state_id
0,CA-2014-100006,2014-09-07,2014-09-13,Standard Class,DK-13375,Dennis Kane,Consumer,United States,New York City,New York,...,TEC-PH-10002075,Technology,Phones,AT&T EL51110 DECT,0.0,377.970,3,109.6113,1,1
1,CA-2014-100090,2014-07-08,2014-07-12,Standard Class,EB-13705,Ed Braxton,Corporate,United States,San Francisco,California,...,FUR-TA-10003715,Furniture,Tables,Hon 2111 Invitation Series Corner Table,0.2,502.488,3,-87.9354,2,2
2,CA-2014-100090,2014-07-08,2014-07-12,Standard Class,EB-13705,Ed Braxton,Corporate,United States,San Francisco,California,...,OFF-BI-10001597,Office Supplies,Binders,"Wilson Jones Ledger-Size, Piano-Hinge Binder, ...",0.2,196.704,6,68.8464,2,2
3,CA-2014-100293,2014-03-14,2014-03-18,Standard Class,NF-18475,Neil Französisch,Home Office,United States,Jacksonville,Florida,...,OFF-PA-10000176,Office Supplies,Paper,Xerox 1887,0.2,91.056,6,31.8696,3,3
4,CA-2014-100328,2014-01-28,2014-02-03,Standard Class,JC-15340,Jasper Cacioppo,Consumer,United States,New York City,New York,...,OFF-BI-10000343,Office Supplies,Binders,"Pressboard Covers with Storage Hooks, 9 1/2"" x...",0.2,3.928,1,1.3257,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9981,US-2017-169551,2017-07-07,2017-07-09,First Class,RL-19615,Rob Lucas,Consumer,United States,Philadelphia,Pennsylvania,...,OFF-PA-10004100,Office Supplies,Paper,Xerox 216,0.2,15.552,3,5.4432,97,13
9982,US-2017-169551,2017-07-07,2017-07-09,First Class,RL-19615,Rob Lucas,Consumer,United States,Philadelphia,Pennsylvania,...,OFF-ST-10004835,Office Supplies,Storage,Plastic Stacking Crates & Casters,0.2,13.392,3,1.0044,97,13
9983,US-2017-169551,2017-07-07,2017-07-09,First Class,RL-19615,Rob Lucas,Consumer,United States,Philadelphia,Pennsylvania,...,TEC-AC-10002018,Technology,Accessories,AmazonBasics 3-Button USB Wired Mouse,0.2,16.776,3,4.8231,97,13
9984,US-2017-169551,2017-07-07,2017-07-09,First Class,RL-19615,Rob Lucas,Consumer,United States,Philadelphia,Pennsylvania,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,0.2,527.920,2,85.7870,97,13


In [18]:
# Пробная таблица товаров: product_id, product_name, category, sub-category
df_products = df_agg_merged[['product_id', 'product_name', 'category', 'sub-category', 'discount']].drop_duplicates(keep='first')
#print(df_products.groupby('product_id')['discount'].nunique().max())
#df_products.groupby('product_id').agg({'discount':'nunique'}) # 1 продукт может иметь разную скидку, discount исключён из таблицы
df_products = df_products.drop('discount', axis = 1)
print(len(df_products) - df['product_id'].nunique())
#df_products['product_id'].duplicated().sum()

check_new_df(df_products)
print('product_id is unique: ', df_products['product_id'].nunique() == len(df_products))
forgotten_unique_val_count(df_products)
print('max product_name for product_id: ', df_products.groupby('product_id')['product_name'].nunique().max())
print('max category for product_id: ', df_products.groupby('product_id')['category'].nunique().max())
print('max sub-category for product_id: ', df_products.groupby('product_id')['sub-category'].nunique().max())

df_products[(df_products.duplicated(subset = 'product_id', keep=False))].sort_values('product_id')
#print(df_renamed[(df_renamed.duplicated(subset = 'product_id', keep=False))].sort_values('product_id'))

#cumulative count

2319
rows:  4181
columns:  4
duplicates:  2287
              unique_values
product_id             1862
product_name           1850
category                  3
sub-category             17
product_id is unique:  False
              forgotten_unique_val
product_id                       0
product_name                     0
category                         0
sub-category                     0
max product_name for product_id:  2
max category for product_id:  1
max sub-category for product_id:  1


,product_id,product_name,category,sub-category
157,FUR-BO-10000330,"Sauder Camden County Barrister Bookcase, Plank...",Furniture,Bookcases
6619,FUR-BO-10000330,"Sauder Camden County Barrister Bookcase, Plank...",Furniture,Bookcases
2149,FUR-BO-10000362,Sauder Inglewood Library Bookcases,Furniture,Bookcases
856,FUR-BO-10000362,Sauder Inglewood Library Bookcases,Furniture,Bookcases
1345,FUR-BO-10000362,Sauder Inglewood Library Bookcases,Furniture,Bookcases
...,...,...,...,...
2243,TEC-PH-10004959,Classic Ivory Antique Telephone ZL1810,Technology,Phones
1539,TEC-PH-10004959,Classic Ivory Antique Telephone ZL1810,Technology,Phones
913,TEC-PH-10004977,GE 30524EE4,Technology,Phones
1602,TEC-PH-10004977,GE 30524EE4,Technology,Phones


In [19]:
#Т.к. одному product_id могут соответствовать несколько product_name, добавим version_id для создания дополнительной таблицы product_versions: version_id, product_id, product_name. Добавим version_id в общий датафрейм
df_product_for_id = df_agg_merged[['product_id', 'product_name']].drop_duplicates()
df_product_for_id['version_id'] = df_product_for_id.groupby('product_id').cumcount()+1
df_agg_merged = df_agg_merged.merge(df_product_for_id, on = ['product_id', 'product_name'], how = 'left')
df_agg_merged

,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,...,category,sub-category,product_name,discount,sales,quantity,profit,location_id,state_id,version_id
0,CA-2014-100006,2014-09-07,2014-09-13,Standard Class,DK-13375,Dennis Kane,Consumer,United States,New York City,New York,...,Technology,Phones,AT&T EL51110 DECT,0.0,377.970,3,109.6113,1,1,1
1,CA-2014-100090,2014-07-08,2014-07-12,Standard Class,EB-13705,Ed Braxton,Corporate,United States,San Francisco,California,...,Furniture,Tables,Hon 2111 Invitation Series Corner Table,0.2,502.488,3,-87.9354,2,2,1
2,CA-2014-100090,2014-07-08,2014-07-12,Standard Class,EB-13705,Ed Braxton,Corporate,United States,San Francisco,California,...,Office Supplies,Binders,"Wilson Jones Ledger-Size, Piano-Hinge Binder, ...",0.2,196.704,6,68.8464,2,2,1
3,CA-2014-100293,2014-03-14,2014-03-18,Standard Class,NF-18475,Neil Französisch,Home Office,United States,Jacksonville,Florida,...,Office Supplies,Paper,Xerox 1887,0.2,91.056,6,31.8696,3,3,1
4,CA-2014-100328,2014-01-28,2014-02-03,Standard Class,JC-15340,Jasper Cacioppo,Consumer,United States,New York City,New York,...,Office Supplies,Binders,"Pressboard Covers with Storage Hooks, 9 1/2"" x...",0.2,3.928,1,1.3257,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9981,US-2017-169551,2017-07-07,2017-07-09,First Class,RL-19615,Rob Lucas,Consumer,United States,Philadelphia,Pennsylvania,...,Office Supplies,Paper,Xerox 216,0.2,15.552,3,5.4432,97,13,1
9982,US-2017-169551,2017-07-07,2017-07-09,First Class,RL-19615,Rob Lucas,Consumer,United States,Philadelphia,Pennsylvania,...,Office Supplies,Storage,Plastic Stacking Crates & Casters,0.2,13.392,3,1.0044,97,13,1
9983,US-2017-169551,2017-07-07,2017-07-09,First Class,RL-19615,Rob Lucas,Consumer,United States,Philadelphia,Pennsylvania,...,Technology,Accessories,AmazonBasics 3-Button USB Wired Mouse,0.2,16.776,3,4.8231,97,13,1
9984,US-2017-169551,2017-07-07,2017-07-09,First Class,RL-19615,Rob Lucas,Consumer,United States,Philadelphia,Pennsylvania,...,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,0.2,527.920,2,85.7870,97,13,1


In [20]:
# Таблица скидок: state_id, product_id, discount
# Т.к. в таблице products уникальный ключ - комбинация product_id и version_id, полный ключ будет использован в таблице promotions
df_promotions = df_agg_merged[['state_id', 'product_id', 'discount']]
print(df_promotions.groupby(['state_id', 'product_id']).agg({'discount': 'nunique'}).max()==1)

df_promotions = df_promotions.drop_duplicates(keep='first')
#df_promotions.groupby('product_id').agg({'state_id': 'nunique', 'discount': 'nunique'}).query('state_id < discount')
print(len(df_promotions)==df_agg_merged[['state_id', 'product_id', 'discount']].groupby(['state_id', 'product_id', 'discount']).count())

#Проверка
check_new_df(df_promotions)
forgotten_unique_val_count(df_promotions.drop(['state_id'], axis=1))

discount    True
dtype: bool
Empty DataFrame
Columns: []
Index: [(1, FUR-BO-10000362, 0.2), (1, FUR-BO-10000780, 0.2), (1, FUR-BO-10001519, 0.2), (1, FUR-BO-10001608, 0.2), (1, FUR-BO-10001811, 0.2), (1, FUR-BO-10001918, 0.2), (1, FUR-BO-10002213, 0.2), (1, FUR-BO-10002268, 0.2), (1, FUR-BO-10002545, 0.2), (1, FUR-BO-10002598, 0.2), (1, FUR-BO-10002613, 0.2), (1, FUR-BO-10003034, 0.2), (1, FUR-BO-10003159, 0.2), (1, FUR-BO-10003272, 0.2), (1, FUR-BO-10003441, 0.2), (1, FUR-BO-10004015, 0.2), (1, FUR-BO-10004695, 0.2), (1, FUR-BO-10004709, 0.2), (1, FUR-BO-10004834, 0.2), (1, FUR-CH-10000225, 0.1), (1, FUR-CH-10000422, 0.1), (1, FUR-CH-10000454, 0.1), (1, FUR-CH-10000553, 0.1), (1, FUR-CH-10000595, 0.1), (1, FUR-CH-10000665, 0.1), (1, FUR-CH-10000785, 0.1), (1, FUR-CH-10000988, 0.1), (1, FUR-CH-10001146, 0.1), (1, FUR-CH-10001215, 0.1), (1, FUR-CH-10001394, 0.1), (1, FUR-CH-10001482, 0.1), (1, FUR-CH-10001545, 0.1), (1, FUR-CH-10001708, 0.1), (1, FUR-CH-10001797, 0.1), (1, FUR-CH-100018

In [21]:
# Таблица клиентов: customer_id, customer_name, segment
df_customers = df_agg_merged[['customer_id', 'customer_name', 'segment']]
df_customers = df_customers.drop_duplicates(keep='first')

print('max customer id for customer name: ', df_customers.groupby('customer_id')['customer_name'].nunique().max())
check_new_df(df_customers)
print('customer_id is unique: ', df_customers['customer_id'].nunique() == len(df_customers))
forgotten_unique_val_count(df_customers)
#df_customers[df_customers.duplicated(subset='customer_id', keep=False)]
df_customers

max customer id for customer name:  1
rows:  793
columns:  3
duplicates:  0
               unique_values
customer_id              793
customer_name            793
segment                    3
customer_id is unique:  True
               forgotten_unique_val
customer_id                       0
customer_name                     0
segment                           0


,customer_id,customer_name,segment
0,DK-13375,Dennis Kane,Consumer
1,EB-13705,Ed Braxton,Corporate
3,NF-18475,Neil Französisch,Home Office
4,JC-15340,Jasper Cacioppo,Consumer
5,JM-15655,Jim Mitchum,Corporate
...,...,...,...
6743,MK-18160,Mike Kennedy,Consumer
7524,CS-12460,Chuck Sachs,Consumer
8242,RM-19750,Roland Murray,Consumer
8690,MO-17950,Michael Oakman,Consumer


In [22]:
print(df_agg_merged.version_id.isnull().sum())
print(df_agg_merged.version_id.isna().sum())
print(df_agg_merged['version_id'].describe())

0
0
count    9986.000000
mean        1.014620
std         0.120034
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         2.000000
Name: version_id, dtype: float64


In [23]:
#Таблица версий товаров
df_product_versions = df_agg_merged[['product_id', 'version_id', 'product_name']]
check_new_df(df_product_versions)
df_product_versions = df_product_versions.drop_duplicates()
print('product_id+version_id is unique: ', len(df_product_versions[['product_id', 'version_id']].drop_duplicates()) == len(df_product_versions))
forgotten_unique_val_count(df_product_versions.drop('version_id', axis=1))

# Итоговая таблица товаров
df_products = df_agg_merged[['product_id', 'category', 'sub-category']].drop_duplicates(keep='first')
print(len(df_products) - df['product_id'].nunique())
check_new_df(df_products)
print('product_id is unique: ', len(df_products[['product_id']]) == len(df_products))
forgotten_unique_val_count(df_products)

rows:  9986
columns:  3
duplicates:  8092
              unique_values
product_id             1862
version_id                2
product_name           1850
product_id+version_id is unique:  True
              forgotten_unique_val
product_id                       0
product_name                     0
0
rows:  1862
columns:  3
duplicates:  0
              unique_values
product_id             1862
category                  3
sub-category             17
product_id is unique:  True
              forgotten_unique_val
product_id                       0
category                         0
sub-category                     0


In [24]:
def quick_check(df, name, key=None):
    """Быстрая проверка DataFrame перед загрузкой в БД"""
    print(f"\n {name}: {len(df)} строк, {len(df.columns)} колонок")
    print(f"   Пропуски: {df.isnull().sum().sum()}")
    print(f"   Дубликаты строк: {df.duplicated().sum()}")
    
    if key:
        key_dups = df.duplicated(subset=key).sum()
        print(f"   Дубликаты по ключу {key}: {key_dups}")
        print(f"   Ключ уникален: {key_dups == 0}")

In [25]:
quick_check(df_product_versions, [['product_id', 'version_id']])


 [['product_id', 'version_id']]: 1894 строк, 3 колонок
   Пропуски: 0
   Дубликаты строк: 0


In [26]:
check_new_df(df_agg_merged)
#df_agg_merged

rows:  9986
columns:  23
duplicates:  0
               unique_values
order_id                5009
order_date              1237
ship_date               1334
ship_mode                  4
customer_id              793
customer_name            793
segment                    3
country                    1
city                     531
state                     49
postal_code              631
region                     4
product_id              1862
category                   3
sub-category              17
product_name            1850
discount                  12
sales                   5826
quantity                  16
profit                  7284
location_id              632
state_id                  49
version_id                 2


(None, None, None, None)

In [27]:
# Таблица заказов-позиций, предположительно: order_id, product_id, version_id, customer_id, location_id, sales, quantity, discount, profit
# Должно быть 7 столбцов из 22 (остальные в справочниках)

#print('Макс. число заказов у одного покупателя: ', df_agg_merged[['customer_id', 'order_id']].groupby('customer_id').agg({'order_id': #'nunique'}).max())
#print('Макс. число владельцев одного заказа: ', df_agg_merged[['customer_id', 'order_id']].groupby('order_id').agg({'customer_id': #'nunique'}).max())
# Т.к. один заказ принадлежит только одному покупателю, customer_id будет добавлен в таблицу orders, а не order_products

df_order_products = df_agg_merged[['order_id', 'product_id', 'version_id', 'location_id', 'sales', 'quantity', 'profit']]
print(df_order_products.duplicated(subset = ['order_id', 'product_id']).sum())
print(len(df_agg))
check_new_df(df_order_products)
forgotten_unique_val_count(df_order_products.drop(['location_id', 'version_id'], axis = 1))
df_order_products

# Уникальные значения sales, quantity, profit могут отличаться по количеству из-за агрегации, доп. проверка:
df_check_oi = pd.DataFrame({'df': (df[['sales', 'quantity', 'profit']].sum()),
              'df_order_products': (df_order_products[['sales', 'quantity', 'profit']].sum())})
df_check_oi['comparing'] = df_check_oi['df'] - df_check_oi['df_order_products']
df_check_oi

0
9986
rows:  9986
columns:  7
duplicates:  0
             unique_values
order_id              5009
product_id            1862
version_id               2
location_id            632
sales                 5826
quantity                16
profit                7284
            forgotten_unique_val
order_id                       0
product_id                     0
sales                          1
quantity                       2
profit                        -3


,df,df_order_products,comparing
sales,2.296919e+06,2.296919e+06,0.0
quantity,3.787100e+04,3.787100e+04,0.0
profit,2.864091e+05,2.864091e+05,0.0


In [28]:
# Создадим справочную таблицу ship_modes: 
df_ship_modes = pd.DataFrame({'ship_mode': df_agg_merged['ship_mode'].unique(), 'ship_mode_id': range(1, df_agg_merged['ship_mode'].nunique()+1)})
df_ship_modes

,ship_mode,ship_mode_id
0,Standard Class,1
1,Second Class,2
2,First Class,3
3,Same Day,4


In [29]:
# Добавим ключ ship_mode_id в датасет
df_agg_merged = df_agg.merge(df_ship_modes[['ship_mode_id', 'ship_mode']], how = 'left')
print(df_agg_merged['ship_mode_id'].nunique()==len(df_ship_modes))
df_agg_merged

True


,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,...,region,product_id,category,sub-category,product_name,discount,sales,quantity,profit,ship_mode_id
0,CA-2014-100006,2014-09-07,2014-09-13,Standard Class,DK-13375,Dennis Kane,Consumer,United States,New York City,New York,...,East,TEC-PH-10002075,Technology,Phones,AT&T EL51110 DECT,0.0,377.970,3,109.6113,1
1,CA-2014-100090,2014-07-08,2014-07-12,Standard Class,EB-13705,Ed Braxton,Corporate,United States,San Francisco,California,...,West,FUR-TA-10003715,Furniture,Tables,Hon 2111 Invitation Series Corner Table,0.2,502.488,3,-87.9354,1
2,CA-2014-100090,2014-07-08,2014-07-12,Standard Class,EB-13705,Ed Braxton,Corporate,United States,San Francisco,California,...,West,OFF-BI-10001597,Office Supplies,Binders,"Wilson Jones Ledger-Size, Piano-Hinge Binder, ...",0.2,196.704,6,68.8464,1
3,CA-2014-100293,2014-03-14,2014-03-18,Standard Class,NF-18475,Neil Französisch,Home Office,United States,Jacksonville,Florida,...,South,OFF-PA-10000176,Office Supplies,Paper,Xerox 1887,0.2,91.056,6,31.8696,1
4,CA-2014-100328,2014-01-28,2014-02-03,Standard Class,JC-15340,Jasper Cacioppo,Consumer,United States,New York City,New York,...,East,OFF-BI-10000343,Office Supplies,Binders,"Pressboard Covers with Storage Hooks, 9 1/2"" x...",0.2,3.928,1,1.3257,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9981,US-2017-169551,2017-07-07,2017-07-09,First Class,RL-19615,Rob Lucas,Consumer,United States,Philadelphia,Pennsylvania,...,East,OFF-PA-10004100,Office Supplies,Paper,Xerox 216,0.2,15.552,3,5.4432,3
9982,US-2017-169551,2017-07-07,2017-07-09,First Class,RL-19615,Rob Lucas,Consumer,United States,Philadelphia,Pennsylvania,...,East,OFF-ST-10004835,Office Supplies,Storage,Plastic Stacking Crates & Casters,0.2,13.392,3,1.0044,3
9983,US-2017-169551,2017-07-07,2017-07-09,First Class,RL-19615,Rob Lucas,Consumer,United States,Philadelphia,Pennsylvania,...,East,TEC-AC-10002018,Technology,Accessories,AmazonBasics 3-Button USB Wired Mouse,0.2,16.776,3,4.8231,3
9984,US-2017-169551,2017-07-07,2017-07-09,First Class,RL-19615,Rob Lucas,Consumer,United States,Philadelphia,Pennsylvania,...,East,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,0.2,527.920,2,85.7870,3


In [30]:
# Создаем таблицу заказов (с предварительной проверкой соответствия одному заказу одной даты заказа, даты поставки, типа доставки, скидки), предположительно: order_id, order_date, ship_date, ship_mode_id
df_orders = df_agg_merged[['order_id', 'customer_id', 'order_date', 'ship_date', 'ship_mode_id']]
print('order_date check: ', df_orders.groupby('order_id')['order_date'].nunique().max())
print('ship_date check: ', df_orders.groupby('order_id')['ship_date'].nunique().max())
print('ship_mode_id check: ', df_orders.groupby('order_id')['ship_mode_id'].nunique().max())
#df_order_check = df_orders.groupby('order_id').agg({'order_date': 'nunique', 'ship_date': 'nunique', 'ship_mode_id': 'nunique', 'discount': 'nunique'})
#print(df_order_check[(df_order_check['order_date'] > 1) | (df_order_check['ship_date'] > 1) | (df_order_check['ship_mode'] > 1) | (df_order_check['discount'] > 1)])
#df_agg_merged[(df_agg_merged['order_id'] == 'CA-2014-100678')]
df_orders = df_agg_merged[['order_id', 'customer_id', 'order_date', 'ship_date', 'ship_mode_id']].drop_duplicates()
check_new_df(df_orders)
forgotten_unique_val_count(df_orders.drop('ship_mode_id', axis=1))
df_orders

order_date check:  1
ship_date check:  1
ship_mode_id check:  1
rows:  5009
columns:  5
duplicates:  0
              unique_values
order_id               5009
customer_id             793
order_date             1237
ship_date              1334
ship_mode_id              4
             forgotten_unique_val
order_id                        0
customer_id                     0
order_date                      0
ship_date                       0


,order_id,customer_id,order_date,ship_date,ship_mode_id
0,CA-2014-100006,DK-13375,2014-09-07,2014-09-13,1
1,CA-2014-100090,EB-13705,2014-07-08,2014-07-12,1
3,CA-2014-100293,NF-18475,2014-03-14,2014-03-18,1
4,CA-2014-100328,JC-15340,2014-01-28,2014-02-03,1
5,CA-2014-100363,JM-15655,2014-04-08,2014-04-15,1
...,...,...,...,...,...
9973,US-2017-168802,JO-15145,2017-11-03,2017-11-07,1
9974,US-2017-169320,LH-16900,2017-07-23,2017-07-25,2
9976,US-2017-169488,AA-10375,2017-09-07,2017-09-09,3
9978,US-2017-169502,MG-17650,2017-08-28,2017-09-01,1


In [31]:
min(df_agg_merged['ship_date'].min(), df_agg_merged['order_date'].min())
# Данные начинаются с 3 января 2014 года

Timestamp('2014-01-03 00:00:00')

In [32]:
# Создадим справочную таблицу dates
df_dates = pd.DataFrame(columns = ['date_id', 'date_day', 'date_month', 'date_week', 'date_quarter', 'date_year', 'day_number_of_week', 'is_weekend'])
df_dates['date_id'] = pd.date_range(datetime.date(2014, 1, 1), datetime.date(2031, 1, 1))
df_dates['date_day'] = df_dates['date_id'].dt.strftime('%d').astype(int)
df_dates['date_week'] = df_dates['date_id'].dt.strftime('%W').astype(int) + 1
df_dates['date_month'] = df_dates['date_id'].dt.strftime('%m').astype(int)
df_dates['date_quarter'] = [1 if 1 <= x < 4 else 2 if 4<=x<7 else 3 if 7<=x<10 else 4 for x in df_dates['date_month']]
df_dates['date_year'] = df_dates['date_id'].dt.strftime('%Y').astype(int)
df_dates['day_number_of_week'] = df_dates['date_id'].dt.dayofweek.astype(int)
df_dates['is_weekend'] = [1 if x in [6, 7] else 0 for x in df_dates['day_number_of_week']]


#df['категория'] = ['A' if x > 30 else 'B' for x in df['возраст']]

df_dates

,date_id,date_day,date_month,date_week,date_quarter,date_year,day_number_of_week,is_weekend
0,2014-01-01,1,1,1,1,2014,2,0
1,2014-01-02,2,1,1,1,2014,3,0
2,2014-01-03,3,1,1,1,2014,4,0
3,2014-01-04,4,1,1,1,2014,5,0
4,2014-01-05,5,1,1,1,2014,6,1
...,...,...,...,...,...,...,...,...
6205,2030-12-28,28,12,52,4,2030,5,0
6206,2030-12-29,29,12,52,4,2030,6,1
6207,2030-12-30,30,12,53,4,2030,0,0
6208,2030-12-31,31,12,53,4,2030,1,0


In [33]:
columns_list = []
columns_list.extend(df_locations.columns)
columns_list.extend(df_states.columns)
columns_list.extend(df_promotions.columns)
columns_list.extend(df_products.columns)
columns_list.extend(df_product_versions.columns)
columns_list.extend(df_customers.columns)
columns_list.extend(df_order_products)
columns_list.extend(df_orders)
columns_list.extend(df_ship_modes)
columns_list = list(set(columns_list))
print(len(columns_list))
print(df_agg_merged.shape[1])
print(list(df_agg_merged.columns))
print([item for item in list(df_agg_merged.columns) if item not in columns_list])
print([item for item in columns_list if item not in list(df_agg_merged.columns)])
columns_list


24
21
['order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id', 'customer_name', 'segment', 'country', 'city', 'state', 'postal_code', 'region', 'product_id', 'category', 'sub-category', 'product_name', 'discount', 'sales', 'quantity', 'profit', 'ship_mode_id']
[]
['version_id', 'state_id', 'location_id']


['quantity',
 'version_id',
 'customer_name',
 'customer_id',
 'state_id',
 'sales',
 'region',
 'discount',
 'ship_mode_id',
 'ship_mode',
 'category',
 'product_id',
 'state',
 'postal_code',
 'city',
 'segment',
 'order_date',
 'product_name',
 'country',
 'sub-category',
 'order_id',
 'ship_date',
 'location_id',
 'profit']

## 2. Загрузка датафреймов в БД Superstore в MS SQL Server

In [39]:
import pyodbc
import pandas as pd

#SERVER = 'My-laptop\\SQLEXPRESS'  # Обрати внимание на двойной слеш
#DATABASE = 'SuperstoreDB'

#conn_str = (
#    f'DRIVER={{ODBC Driver 17 for SQL Server}};'
#    f'SERVER={SERVER};'
#    f'DATABASE={DATABASE};'
#    f'Trusted_Connection=yes;'
#)

#conn = pyodbc.connect(conn_str)


# Параметры подключения для БД
SERVER = 'My-laptop\\SQLEXPRESS' #нужен двойной \
DATABASE = 'Superstore'

conn_str = (
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'
    f'SERVER={SERVER};'
    f'DATABASE={DATABASE};'
    f'Trusted_Connection=yes;'
)

conn = pyodbc.connect(conn_str)

#conn = pyodbc.connect(
#    'DRIVER={ODBC Driver 17 for SQL Server};'
#    'SERVER=My-laptop\SQLEXPRESS;'
#    'DATABASE=SuperstoreDB;'
#    'Trusted_Connection=yes;'
#)
cursor = conn.cursor()

for index, row in df_customers.iterrows():
    cursor.execute(
        "INSERT INTO dbo.Customers (CustomerID, CustomerName, Segment) VALUES (?,?,?)",
        row.customer_id, row.customer_name, row.segment
    )
conn.commit()
print('Customers uploaded')

for index, row in df_dates.iterrows():
    cursor.execute(
        "INSERT INTO dbo.Dates (DateID, DateDay, DateWeek, DateMonth, DateQuarter, DateYear, IsWeekend, DayNumberOfWeek) VALUES (?,?,?,?,?,?,?,?)",
        row.date_id, row.date_day, row.date_week, row.date_month, row.date_quarter, row.date_year, row.is_weekend, row.day_number_of_week
    )
conn.commit()
print('Dates uploaded')

for index, row in df_products.iterrows():
    cursor.execute(
        "INSERT INTO dbo.Products (ProductID, Category, SubCategory) VALUES (?,?,?)",
        row.product_id, row.category, row['sub-category']
    )
conn.commit()
print('Products uploaded')

for index, row in df_product_versions.iterrows():
    cursor.execute(
        "INSERT INTO dbo.ProductVersions (ProductID, VersionID, ProductName) VALUES (?,?,?)",
        row.product_id, row.version_id, row.product_name
    )
conn.commit()
print('ProductVersions uploaded')

for index, row in df_states.iterrows():
    cursor.execute(
        "INSERT INTO dbo.States (StateID, StateName, Region, Country) VALUES (?,?,?, ?)",
        row.state_id, row.state, row.region, row.country
    )
conn.commit()
print('States uploaded')

for index, row in df_locations.iterrows():
    cursor.execute(
        "INSERT INTO dbo.Locations (LocationID, StateID, City, PostalCode) VALUES (?,?,?,?)",
        row.location_id, row.state_id, row.city, row.postal_code
    )
conn.commit()
print('Locations uploaded')

for index, row in df_promotions.iterrows():
    cursor.execute(
        "INSERT INTO dbo.Promotions (StateID, ProductID, Discount) VALUES (?,?,?)",
        row.state_id, row.product_id, row.discount
    )
conn.commit()
print('Promotions uploaded')

for index, row in df_ship_modes.iterrows():
    cursor.execute(
        "INSERT INTO dbo.ShipModes (ShipModeID, ShipMode) VALUES (?,?)",
        row.ship_mode_id, row.ship_mode
    )
conn.commit()
print('ShipModes uploaded')

for index, row in df_orders.iterrows():
    cursor.execute(
        "INSERT INTO dbo.Orders (OrderID, CustomerID, OrderDate, ShipDate, ShipModeID) VALUES (?,?,?,?,?)",
        row.order_id, row.customer_id, row.order_date, row.ship_date, row.ship_mode_id
    )
conn.commit()
print('Orders uploaded')

for index, row in df_order_products.iterrows():
    cursor.execute(
        "INSERT INTO dbo.OrderProducts (OrderID, ProductID, VersionID, LocationID, Sales, Quantity, Profit) VALUES (?,?,?,?,?,?,?)",
        row.order_id, row.product_id, row.version_id, row.location_id, row.sales, row.quantity, row.profit
    )
conn.commit()
print('OrderProducts uploaded')

cursor.close()
conn.close()

Customers uploaded
Dates uploaded
Products uploaded
ProductVersions uploaded
States uploaded
Locations uploaded
Promotions uploaded
ShipModes uploaded
Orders uploaded
OrderProducts uploaded


In [ ]:
# df_product_versions[df_product_versions['product_id'] == 'OFF-AR-10004757']

In [40]:
df_order_products

,order_id,product_id,version_id,location_id,sales,quantity,profit
0,CA-2014-100006,TEC-PH-10002075,1,1,377.970,3,109.6113
1,CA-2014-100090,FUR-TA-10003715,1,2,502.488,3,-87.9354
2,CA-2014-100090,OFF-BI-10001597,1,2,196.704,6,68.8464
3,CA-2014-100293,OFF-PA-10000176,1,3,91.056,6,31.8696
4,CA-2014-100328,OFF-BI-10000343,1,1,3.928,1,1.3257
...,...,...,...,...,...,...,...
9981,US-2017-169551,OFF-PA-10004100,1,97,15.552,3,5.4432
9982,US-2017-169551,OFF-ST-10004835,1,97,13.392,3,1.0044
9983,US-2017-169551,TEC-AC-10002018,1,97,16.776,3,4.8231
9984,US-2017-169551,TEC-AC-10003033,1,97,527.920,2,85.7870
